In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
import math
import tqdm
import numpy as np
import pandas as pd

torch.manual_seed(192837)
rand = random.Random(192838)

In [2]:
CITY_COLUMN = 'city_ascii'
COUNTRY_COLUMN = 'iso2'

In [3]:
df_all = pd.read_csv('../emb/data/worldcities.csv')

'''
df_top_countries = df_all.groupby(COUNTRY_COLUMN).size().sort_values(ascending=False).reset_index()
classes = list(df_top_countries.head(N_CLASSES)[COUNTRY_COLUMN])
'''
classes = ['IN']
N_CLASSES = len(classes)

print(classes)
class_to_id = {classes[i]: i for i in range(N_CLASSES)}

df_all = df_all[df_all[COUNTRY_COLUMN].isin(classes)]
df_all = df_all[df_all[CITY_COLUMN].notna()]
df_all.head(10)

['IN']


,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629
10,Kolkāta,Kolkata,22.5675,88.3700,India,IN,IND,West Bengal,admin,21747000.0,1356060520
21,Bangalore,Bangalore,12.9789,77.5917,India,IN,IND,Karnātaka,admin,15386000.0,1356410365
29,Chennai,Chennai,13.0825,80.2750,India,IN,IND,Tamil Nādu,admin,12395000.0,1356374944
39,Hyderābād,Hyderabad,17.3617,78.4747,India,IN,IND,Telangāna,admin,10494000.0,1356871768
55,Pune,Pune,18.5203,73.8567,India,IN,IND,Mahārāshtra,NaN,8231000.0,1356081074
58,Ahmedabad,Ahmedabad,23.0225,72.5714,India,IN,IND,Gujarāt,minor,8009000.0,1356304381
79,Sūrat,Surat,21.2050,72.8400,India,IN,IND,Gujarāt,NaN,6538000.0,1356758738
90,Prayagraj,Prayagraj,25.4358,81.8464,India,IN,IND,Uttar Pradesh,NaN,5954391.0,1356718332


In [4]:
character_set = set()
for city in df_all[CITY_COLUMN]:
    for ch in city:
        character_set.add(ch)

PAD_TOKEN = 0
tokens = ['<PAD>'] + sorted(list(character_set))
token_to_id = {tokens[i]: i for i in range(len(tokens))}
N_VOCAB = len(tokens)

''.join(tokens)

'<PAD> -ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz'

In [5]:
def encode(city: str, max_length: int):
    encoded = [token_to_id[city[i]] for i in range(min(max_length, len(city)))]
    return encoded + [PAD_TOKEN] * (max_length - len(encoded))

def decode(city_enc: list[int], decode_special: bool=False):
    return ''.join([tokens[tk] for tk in city_enc if (tk != PAD_TOKEN or decode_special)])

In [6]:
city_length_90_perc = np.quantile(np.array(list(map(len, df_all[CITY_COLUMN]))), 0.8)

SEQ_LENGTH = int(city_length_90_perc)
SEQ_LENGTH

11

In [7]:
data_list_x = [encode(city, SEQ_LENGTH) for city in df_all[CITY_COLUMN]]
data_list_y = [class_to_id[iso2] for iso2 in df_all[COUNTRY_COLUMN]]
data_list = list(zip(data_list_x, data_list_y))
rand.shuffle(data_list)

data_x = torch.tensor(np.array([x for x, _ in data_list], dtype=np.long))
data_y = torch.tensor(np.array([y for _, y in data_list], dtype=np.long))

for i in range(5):
    print(decode(data_x[i]), classes[data_y[i]])

TRAIN_TEST_SPLIT = 0.9
n_train = round(data_x.shape[0] * TRAIN_TEST_SPLIT)
train_x, train_y = data_x[:n_train, :], data_y[:n_train]
test_x,  test_y  = data_x[n_train:, :], data_y[n_train:]

print(data_x.shape[0], n_train, data_x.shape[0] - n_train)

Barah IN
Chittaurgar IN
Bhogpur IN
Jami IN
Bhadreswar IN
7108 6397 711


In [8]:
class RNNGenerator(nn.Module):
    def __init__(self,
                 emb_dim: int,
                 hidden_dim: int,
                 hidden_layers: int=1):
        super().__init__()

        self.emb = nn.Linear(N_VOCAB, emb_dim, bias=False)

        self.rnn = nn.GRU(emb_dim, hidden_dim, num_layers=hidden_layers, batch_first=True)
        self.head_1 = nn.Linear(hidden_dim, hidden_dim)
        self.head_2 = nn.Linear(hidden_dim, hidden_dim)
        self.head_3 = nn.Linear(hidden_dim, N_VOCAB)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor):
        if len(x.shape) == 2:  # unbatched input
            x = x.unsqueeze(0)
        B, *_ = x.shape

        x = self.emb(x)
        rnn_o, h_n = self.rnn(x)
        o = self.relu(self.head_1(rnn_o))
        o = self.relu(self.head_2(o))
        o = self.head_3(o)
        return o

In [9]:
def train_epoch(model: nn.Module,
                optimizer: optim.Optimizer,
                dataset_x: torch.Tensor,
                batch_size: int):
    model.train()
    
    loss_sum = 0
    accu_sum = 0
    n_batches = math.ceil(dataset_x.shape[0] / batch_size)

    for i in tqdm.tqdm(range(n_batches), 'train'):
        x = dataset_x[i * batch_size: (i+1) * batch_size, :]
        x_oh = F.one_hot(x, N_VOCAB).to(dtype=torch.float32)
        x_hat = model.forward(x_oh)

        # DEBUG
        # print(x.shape, x_oh.shape, x_hat.shape)

        loss = F.cross_entropy(x_hat[:, :-1, :].reshape((-1, N_VOCAB)), x[:, 1:].reshape((-1,)), ignore_index=PAD_TOKEN)
        loss_sum += loss.detach().clone()

        with torch.no_grad():
            accu_sum += torch.sum(x_hat[:, :-1, :].argmax(dim=-1) == x[:, 1:]) / x.shape[0] / (x.shape[1] - 1)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    return loss_sum / n_batches, accu_sum / n_batches

def test_epoch(model: nn.Module, dataset_x: torch.Tensor, batch_size):
    model.eval()
    
    with torch.no_grad():
        loss_sum = 0
        accu_sum = 0
        n_batches = math.ceil(dataset_x.shape[0] / batch_size)

        for i in tqdm.tqdm(range(n_batches), ' test'):
            x = dataset_x[i * batch_size: (i+1) * batch_size, :]
            x_oh = F.one_hot(x, N_VOCAB).to(dtype=torch.float32)
            x_hat = model.forward(x_oh)

            loss = F.cross_entropy(x_hat[:, :-1, :].reshape((-1, N_VOCAB)), x[:, 1:].reshape((-1,)), ignore_index=PAD_TOKEN)
            loss_sum += loss
            accu_sum += torch.sum(x_hat[:, :-1, :].argmax(dim=-1) == x[:, 1:]) / x.shape[0] / (x.shape[1] - 1)

        return loss_sum / n_batches, accu_sum / n_batches

In [10]:
model = RNNGenerator(emb_dim=16,
                     hidden_dim=32,
                     hidden_layers=2)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-5)
N_EPOCHS = 50
BATCH_SIZE = 32

In [11]:
for epoch_i in range(N_EPOCHS):
    print(f'=== epoch {epoch_i} ===')

    train_loss, train_accu = train_epoch(model, optimizer, train_x, BATCH_SIZE)
    test_loss, test_accu = test_epoch(model, test_x, BATCH_SIZE)

    print('train loss:', train_loss.item())
    print(' test loss:', test_loss.item())
    print('train accu:', train_accu.item())
    print(' test accu:', test_accu.item())

=== epoch 0 ===


 test: 100%|██████████| 23/23 [00:00<00:00, 373.38it/s]


train loss: 3.0323071479797363
 test loss: 2.7356069087982178
train accu: 0.1586713343858719
 test accu: 0.18144410848617554
=== epoch 1 ===


 test: 100%|██████████| 23/23 [00:00<00:00, 372.14it/s]


train loss: 2.5660009384155273
 test loss: 2.3441948890686035
train accu: 0.20120632648468018
 test accu: 0.23051242530345917
=== epoch 2 ===


...

train loss: 1.9272695779800415
 test loss: 1.9765784740447998
train accu: 0.2894498407840729
 test accu: 0.284297376871109
=== epoch 48 ===


 test: 100%|██████████| 23/23 [00:00<00:00, 370.96it/s]


train loss: 1.923789381980896
 test loss: 1.9746588468551636
train accu: 0.2900296151638031
 test accu: 0.2837538421154022
=== epoch 49 ===


 test: 100%|██████████| 23/23 [00:00<00:00, 372.56it/s]

train loss: 1.9203715324401855
 test loss: 1.9723210334777832
train accu: 0.29088738560676575
 test accu: 0.2835403382778168


In [12]:
from torch.distributions import Categorical

for test in ('Mosc', 'Rio ', 'Delh', 'Shan'):
    test_tokens = encode(test, 4)
    test_tokens = torch.tensor(test_tokens, dtype=torch.long).unsqueeze(0)

    for i in range(4, SEQ_LENGTH):
        pred = model.forward(F.one_hot(test_tokens, N_VOCAB).to(torch.float32))
        dist = Categorical(logits=pred[:, -1, :])
        next_tokens = dist.sample().unsqueeze(1)

        test_tokens = torch.cat((test_tokens, next_tokens), dim=-1)

    print(decode(test_tokens.squeeze(), decode_special=True))

Moschaurika
Rio Manjunk
Delhadinghe
Shaniharcha


In [13]:
from torch.distributions import Categorical
from torch.distributions.gumbel import Gumbel
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

class AntiRNNGen():
    def __init__(self,
                 inner_net: nn.Module,
                 initial_tokens: int=6,
                 desired_tokens: str='abc',
                 lr: float=1e-2,
                 right_c_coef: float=1.0,
                 wrong_c_coef: float=1.0):
        super().__init__()

        self.n_initial_tokens = initial_tokens
        self.n_desired_tokens = len(desired_tokens)
        self.desired_tokens = [token_to_id[ch] for ch in desired_tokens]
        self.desired_tokens = torch.tensor(self.desired_tokens, dtype=torch.long)

        self.input_shape = (1, initial_tokens, N_VOCAB)
        self.input = nn.Parameter(
            data=torch.normal(0, 1, size=self.input_shape),
            requires_grad=True
        )

        self.inner_net = inner_net
        self.grad_eraser = torch.optim.SGD(self.inner_net.parameters())
        self.anti_optimizer = torch.optim.Adam((self.input,), lr=lr)
        # self.anti_scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

        self.right_c_coef = right_c_coef
        self.wrong_c_coef = wrong_c_coef

        # self.input_min = 0.0
        # self.input_max = 1.0

        self.gumbel = Gumbel(loc=0.0, scale=1.0)
        self.epsilon = 1e-8

    def test(self):
        with torch.no_grad():
            return self.inner_net.forward(self.input)

    def zero_grad(self):
        self.grad_eraser.zero_grad()
        self.anti_optimizer.zero_grad()

    # def crop(self):
    #     self.input.data.copy_(torch.clamp(self.input.detach().clone(), self.input_min, self.input_max))
    
    def step(self):
        prev_input = self.input.detach().clone()

        probs = torch.softmax(self.input, dim=-1)

        extra_tokens = torch.zeros((1, self.n_desired_tokens - 1, N_VOCAB))
        extra_tokens[0, torch.arange(0, self.n_desired_tokens - 1), self.desired_tokens[:-1]] = 1.0
        probs = torch.cat((probs, extra_tokens), dim=1)

        gumb = self.gumbel.sample(probs.shape)

        j_onehot = F.one_hot(torch.argmax(gumb + torch.log(probs + self.epsilon), dim=-1), N_VOCAB)
        j_onehot = j_onehot.to(torch.float32).requires_grad_(True)

        j_continuous = torch.softmax(gumb + torch.log(probs + self.epsilon), dim=-1)

        output = self.inner_net.forward(j_onehot)
        output_soft = torch.softmax(output, dim=-1)
        
        # DEBUG
        # print(j_onehot.shape, j_continuous.shape, output.shape, output_soft.shape)

        loss = output_soft[:, self.n_initial_tokens - 1: self.n_initial_tokens + self.n_desired_tokens - 1, :]\
                          [torch.arange(0, N_VOCAB).unsqueeze(0).unsqueeze(1) != self.desired_tokens.unsqueeze(0).unsqueeze(2)]\
            .sum() * self.wrong_c_coef + \
               output_soft[:, self.n_initial_tokens - 1: self.n_initial_tokens + self.n_desired_tokens - 1, :]\
                          [:, :, self.desired_tokens].sum() * -self.right_c_coef
        loss.backward()

        j_continuous.backward(j_onehot.grad)

        self.anti_optimizer.step()
        self.zero_grad()

        # self.anti_scheduler.step()

        # self.crop()

        return prev_input, output.detach().clone(), loss.detach().clone()

In [19]:
ANTI_NET_STEPS = 3000
TEST_TRIES = 4
TESTS = ('pal', 'pur', 'ana', 'tia', 'ish', 'ika', 'ung', 'cha')

for i in range(len(TESTS)):
    n_desired_tokens = 3
    desired_tokens = TESTS[i]

    print(f'-=-=- step {i} -=-=-')
    print(f'desired tokens:', desired_tokens)

    fin_loss = float('+inf')
    fin_inp, fin_outp = None, None

    for try_i in range(TEST_TRIES):
        anti_net = AntiRNNGen(model, initial_tokens=6, desired_tokens=desired_tokens, lr=5e-2)

        for step_i in tqdm.tqdm(range(ANTI_NET_STEPS), f'attempt #{try_i}'):
            inp, outp, loss = anti_net.step()
        
        if loss.item() < fin_loss:
            fin_loss = loss.item()
            fin_inp = inp
            fin_outp = outp
        
        print(f'attempt #{try_i} loss:', loss.item())
    
    print(f'fin loss:', fin_loss)
    print(f'fin input:', decode(fin_inp.argmax(dim=-1).squeeze()))
    print(f'fin output (from function):', decode(fin_outp.argmax(dim=-1).squeeze(), decode_special=True))
    
    outp_gen = fin_inp.argmax(dim=-1)
    for i in range(n_desired_tokens):
        pred = model.forward(F.one_hot(outp_gen, N_VOCAB).to(torch.float32))
        next_tokens = pred[:, -1, :].argmax(dim=-1).unsqueeze(1)
        outp_gen = torch.cat((outp_gen, next_tokens), dim=-1)

    print(f'fin output (argmax sample):', decode(outp_gen.squeeze(), decode_special=True))

    outp_gen = fin_inp.argmax(dim=-1)
    for i in range(n_desired_tokens):
        pred = model.forward(F.one_hot(outp_gen, N_VOCAB).to(torch.float32))

        dist = Categorical(logits=pred[:, -1, :])
        next_tokens = dist.sample().unsqueeze(1)

        outp_gen = torch.cat((outp_gen, next_tokens), dim=-1)

    print(f'fin output (random sample):', decode(outp_gen.squeeze(), decode_special=True))

-=-=- step 0 -=-=-
desired tokens: pal


attempt #0: 100%|██████████| 3000/3000 [00:27<00:00, 110.29it/s]


attempt #0 loss: 1.3960310220718384


attempt #1: 100%|██████████| 3000/3000 [00:26<00:00, 114.04it/s]


attempt #1 loss: -0.5257922410964966


...

attempt #1 loss: -0.4566993713378906


attempt #2: 100%|██████████| 3000/3000 [00:43<00:00, 69.35it/s]


attempt #2 loss: -0.4763883352279663


attempt #3: 100%|██████████| 3000/3000 [00:45<00:00, 65.76it/s]


attempt #3 loss: -0.5307084321975708
fin loss: -0.5307084321975708
fin input: EnlY w
fin output (from function): rdaaraha
fin output (argmax sample): EnlY wall
fin output (random sample): EnlY wani
